In [83]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA

In [ ]:
npors_raw = pd.read_spss("NPORS_2024_for_public_release.sav")

In [87]:
npors_raw2 = npors_raw.iloc[:, 9:]
npors_raw3 = npors_raw.loc[:, ['ECON1MOD', 'EDUCATION', 'PARTY']]
npors_raw3 = npors_raw3.replace("Don't know/Refused/Web blank", np.nan)
npors_raw4 = npors_raw3.dropna()

C:\Users\mbans\AppData\Local\Temp\ipykernel_15368\1775069237.py:3: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  npors_raw3 = npors_raw3.replace("Don't know/Refused/Web blank", np.nan)


In [89]:
encoder = OneHotEncoder()
npors_encoded = encoder.fit_transform(npors_raw4.loc[:, ['ECON1MOD', 'EDUCATION']])
features = encoder.get_feature_names_out()
npors = pd.DataFrame(data=npors_encoded.toarray(), columns=features)

In [90]:
party_encoder = LabelEncoder()
party_encoded = party_encoder.fit_transform(npors_raw4['PARTY'])
npors['PARTY'] = party_encoded

In [91]:
X = npors.drop(['PARTY'], axis=1).to_numpy()
y = npors['PARTY'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [92]:
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [102]:
knn_clf = KNeighborsClassifier(n_neighbors=17, metric='manhattan')
knn_clf.fit(X_train_pca, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=17)

In [103]:
y_pred = knn_clf.predict(X_test_pca)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: ", acc)

clf_rep = classification_report(y_test, y_pred)
print(f"Classification report: {clf_rep}")

Accuracy:  0.38064516129032255
Classification report:               precision    recall  f1-score   support

           0       0.41      0.59      0.49       394
           1       0.00      0.00      0.00       260
           2       0.35      0.57      0.44       319
           3       0.00      0.00      0.00       112

    accuracy                           0.38      1085
   macro avg       0.19      0.29      0.23      1085
weighted avg       0.25      0.38      0.30      1085



C:\Users\mbans\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mbans\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mbans\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [101]:
param_grid = {'n_neighbors': list(range(3, 20)), 'metric': ['manhattan', 'euclidean']}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_train_pca, y_train)
print("Best Parameters: ", grid_search.best_params_)

Best Parameters:  {'metric': 'manhattan', 'n_neighbors': 17}
